In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

# Generate training data
np.random.seed(42)
num_samples = 1000
x_train = np.random.uniform(low=0, high=1, size=(num_samples, 1))
t_train = np.random.uniform(low=0, high=1, size=(num_samples, 1))
u_exact = np.sin(np.pi * x_train) * np.exp(-np.pi**2 * t_train)

# Convert to TensorFlow tensors
x_train_tf = tf.convert_to_tensor(x_train, dtype=tf.float32)
t_train_tf = tf.convert_to_tensor(t_train, dtype=tf.float32)
u_exact_tf = tf.convert_to_tensor(u_exact, dtype=tf.float32)

# Combine x and t for training input
input_train = tf.concat([x_train_tf, t_train_tf], axis=1)

# Generate test data for prediction
x_test = np.linspace(0, 1, 100).reshape(-1, 1)
t_test = np.linspace(0, 1, 100).reshape(-1, 1)

# Convert to TensorFlow tensors
x_test_tf = tf.convert_to_tensor(x_test, dtype=tf.float32)
t_test_tf = tf.convert_to_tensor(t_test, dtype=tf.float32)
input_test = tf.concat([x_test_tf, t_test_tf], axis=1)
u_exact_test = np.sin(np.pi * x_test) * np.exp(-np.pi**2 * t_test)
u_exact_test_tf = tf.convert_to_tensor(u_exact_test, dtype=tf.float32)

In [3]:
# Define the neural network model
model = Sequential([
    Dense(units=50, activation='tanh'),
    Dense(units=50, activation='tanh'),
    Dense(units=1, activation='linear')
])

In [4]:
initial_alpha = tf.Variable(0.00001, dtype=tf.float32)

# Define the loss function
def loss(model, x, t, alpha):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        tape.watch(t)
        u_pred = model(tf.concat((x, t), axis=1))
        du_x = tape.gradient(u_pred, x)
        du_xx = tape.gradient(du_x, x)
        du_dt = tape.gradient(u_pred, t)
        del tape

    physics_residual = du_dt - alpha * du_xx
    physics_loss = tf.reduce_mean(tf.square(physics_residual))
    data_loss = tf.reduce_mean(tf.square(u_pred - u_exact_tf))
    total_loss = physics_loss + data_loss
    
    return total_loss, physics_loss, data_loss

In [5]:
# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Training loop
num_epochs = 2000
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        total_loss, physics_loss_value, data_loss_value = loss(model, x_train_tf, t_train_tf, initial_alpha)
        
    gradients = tape.gradient(total_loss, model.trainable_variables + [initial_alpha])
    optimizer.apply_gradients(zip(gradients, model.trainable_variables + [initial_alpha]))

    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{num_epochs}, Total Loss: {total_loss.numpy()}, Physics Loss: {physics_loss_value.numpy()}, Data Loss: {data_loss_value.numpy()}")


Epoch 0/2000, Total Loss: 0.04439772665500641, Physics Loss: 0.023873697966337204, Data Loss: 0.020524026826024055
Epoch 100/2000, Total Loss: 0.019875263795256615, Physics Loss: 0.0006349727045744658, Data Loss: 0.01924029178917408
Epoch 200/2000, Total Loss: 0.01598675362765789, Physics Loss: 0.000977875548414886, Data Loss: 0.015008878894150257
Epoch 300/2000, Total Loss: 0.01297401450574398, Physics Loss: 0.0008847829885780811, Data Loss: 0.012089231982827187
Epoch 400/2000, Total Loss: 0.011781474575400352, Physics Loss: 0.00044486665865406394, Data Loss: 0.011336607858538628
Epoch 500/2000, Total Loss: 0.010818644426763058, Physics Loss: 0.00022930103295948356, Data Loss: 0.01058934349566698
Epoch 600/2000, Total Loss: 0.010245248675346375, Physics Loss: 0.0003886642516590655, Data Loss: 0.00985658448189497
Epoch 700/2000, Total Loss: 0.0091127073392272, Physics Loss: 0.0002756241010501981, Data Loss: 0.008837083354592323
Epoch 800/2000, Total Loss: 0.00822842214256525, Physics L

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [14]:
# Prediction
u_pred = model(tf.concat([input_test], axis=1))

# Calculate Mean Squared Error
mse = mean_squared_error(u_exact_test_tf, u_pred)
print(f"Mean Squared Error: {mse}")

# Calculate Mean Absolute Error
mae = mean_absolute_error(u_exact_test_tf, u_pred)
print(f"Mean Absolute Error: {mae}")


Mean Squared Error: 0.001226570806466043
Mean Absolute Error: 0.02545933797955513
